## 1. Extract from PDF

In [437]:
import fitz  # PyMuPDF

# ƒê·ªçc file PDF
pdf_file = "Daivietsukytoanthu.pdf"
doc = fitz.open(pdf_file)


In [438]:
title = 'ƒê ·∫°i  V i ·ªát  S ·ª≠ K √Ω'

In [439]:
import re
from collections import defaultdict

def get_content(page):
    note =''
    content =''
    text = page.get_text("text")
    try:
        text = re.split(r"^\d+\s*\n.*\n", text, maxsplit=1, flags=re.MULTILINE)[1]
        text = text.split('\n                                                \n')
        if len(text)>1: note = text[1]
        content = text[0]
    except:
        return text, ''
    
    return content, note

In [440]:
def is_new(event):
    pattern = r'^(\[?[0-9]+[a-z]?\]?\s+)?[^\/\[\]]*(\/?[^\/\[\]]+\/?)?[^\[\]]*(\[[^\[\]]+\])?[^\[\]]*\[\s*[0-9]+\s*(TCN)?\s*\]'
    if re.match(pattern,event):
        return True
    return False

In [441]:
c1,c2 = get_content(doc[125])
c1

'Kh·ªïng Nguy√™n, Ph·∫°m B·∫£o, Kim C√°t, L√Ω Kh√°i, ƒê√†o L√£o, Nguy·ªÖn Ho√†n l√†m Vi√™n ngo·∫°i lang th∆∞·ª£ng th∆∞ \ns·∫£nh; L∆∞∆°ng C·ª≠u, ƒê√†o S√¢m, Qu√°ch Th·ª•c, Nguy·ªÖn Nh√¢n, Nguy·ªÖn Kh√°nh, ƒê√†o T∆∞∆°ng, Qu√°ch C·ª± T·∫ßm, \nNguy·ªÖn Th·ªëi l√†m Trung th∆∞ ho·∫£; L√Ω Ng≈©, Ki·ªÉu Nghƒ©a, L√Ω C√°, Nguy·ªÖn Bi·∫øm, Nguy·ªÖn B·ªôc, Nguy·ªÖn Khoan, \nƒê√†o L·ª•c, ƒê·ªó K√Ω, Ki·ªÉu Thi·ªáu l√†m Chi h·∫≠u th∆∞ gia. \n  \nNg√†y Qu√Ω S·ª≠u, xu·ªëng chi·∫øu cho Gi√°n ngh·ªã ƒë·∫°i phu M√¢u Du ƒê√¥ ch·ªçn qu√¢n Long D·ª±c c≈© l√†m c√°c \nqu√¢n t·∫£ h·ªØu Ng·ªçc Giai, H∆∞ng Th√°nh, Qu·∫£ng Th√†nh, v≈© [30a] ƒê√¥. \n  \nNg√†y Gi√°p D·∫ßn, h∆°n 2 v·∫°n ng∆∞·ªùi Ch√¢n L·∫°p v√†o c∆∞·ªõp b·∫øn Ba ƒê·∫ßu ·ªü ch√¢u Ngh·ªá An. Xu·ªëng chi·∫øu \ncho Nh·∫≠p n·ªôi th√°i ph√≥ L√Ω C√¥ng B√¨nh ƒëem c√°c quan ch·ª©c ƒë√¥ c√πng ng∆∞·ªùi ch√¢u Ngh·ªá An ƒëi ƒë√°nh. \n  \nTh√°ng 2, ng√†y ·∫§t M√£o, xu·ªëng chi·∫øu tha cho c√° t·ªôi nh√¢n ·ªü ph·ªß ƒê√¥ h·ªô. Ng√†y Nh√¢m Tu·∫•t, xu·ªëng \nchi·∫øu th

In [442]:
events = []
page_number = []
event = ''
notes = []
n = 0
for i, page in enumerate(doc):  
    content, note = get_content(page)
    content_split = content.split('\n')
    
    notes.append(note)
    if i% 50 == 0:
        print(i)
    if content_split[0].find(title) == 0:
        content_split = content_split[2:]
        page_number.append(n)
        events.append(event)
        event = ''
        n = i
    for item in content_split:
        if item=='': continue
        if (is_new(item) and event.endswith('. ')) or item[:2]=='K·ª∑':
            page_number.append(n)
            n = i
            events.append(event)
            event = item
        else:
            event = event + '\n' + item



0
50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [740]:
events = events[1:]
page_number = page_number[1:]

In [741]:
len(events)

920

In [574]:
#copy = events
#copy.append('x')
copy = events.copy()

In [739]:
events = copy.copy()

In [492]:
def find_index(target):
    for i, event in enumerate(events):
        if event.find(target) > 0:
            return i, page_number[i] + 1

In [674]:
find_index('Chi√™u Ho√†ng ')

(425, 156)

## 2. Preprocessing

In [702]:
def is_proper_name(text):
    
    # Ki·ªÉm tra xem text c√≥ k·∫øt th√∫c b·∫±ng " \n" kh√¥ng
    if not text.endswith(" "):
        return False
    
    # Lo·∫°i b·ªè " " ·ªü cu·ªëi ƒë·ªÉ ƒë·∫øm s·ªë t·ª´
    content = text[:-1]
    
    # T√°ch c√°c t·ª´ v√† ƒë·∫øm
    words = content.split(' ')
    
    if len(words) < 2: 
        return False
    if len(words) > 7: 
        return False
        
    for word in words:
        word = word.rstrip(',')
        if len(word) <1 : return False
        if word[0] != word[0].upper():
            return False
        if not all(c.isalpha() or c == ',' or c =='(' or c ==')' or c =='_' for c in word):
            return False
        
    return True
        

In [694]:
is_proper_name('H√πng V∆∞∆°ng ')

True

In [743]:
len(events)

953

In [742]:
d = 0
for i, e in enumerate(events):
    for p in e.split('\n'):
        if is_proper_name(p):
            if e.find(p) > 100:
                split1 = e[:e.find(p)]
                split2 = e[e.find(p):]
                events = events[:i + d] + [split1,split2] + events[i + 1 + d:]
                d += 1
                print(p)
                print(e[:].find(p),  i)

L·∫°c Long Qu√¢n 
607 1
H√πng V∆∞∆°ng 
2413 1
VƒÉn V∆∞∆°ng 
1644 43
Minh V∆∞∆°ng 
397 48
Ai V∆∞∆°ng 
560 53
Thu·∫≠t D∆∞∆°ng V∆∞∆°ng 
4186 54
D∆∞∆°ng Tam Kha 
751 171
PH·∫æ ƒê·∫æ 
3833 194
TRUNG T√îNG HO√ÄNG ƒê·∫æ 
1456 208
NG·ªåA TRI·ªÄU HO√ÄNG ƒê·∫æ 
3491 208
TH√ÅI T√îNG HO√ÄNG ƒê·∫æ 
14351 231
NH√ÇN T√îNG HO√ÄNG ƒê·∫æ 
1561 275
TH·∫¶N T√îNG HO√ÄNG ƒê·∫æ 
6364 329
Cao T√¥ng Ho√†ng ƒê·∫ø 
2509 379
Hu·ªá T√¥ng Ho√†ng ƒê·∫ø 
2235 412
NH√ÇN T√îNG HO√ÄNG ƒê·∫æ 
9679 473
MINH T√îNG HO√ÄNG ƒê·∫æ 
1530 504
NGH·ªá T√îNG HO√ÄNG ƒê·∫æ 
5910 555
Du·ªá T√îNG HO√ÄNG ƒê·∫æ 
1688 558
THU·∫¨N T√îNG HO√ÄNG ƒê·∫æ 
6856 574
THI·∫æU ƒê·∫æ 
2354 583
TR√ôNG QUANG ƒê·∫æ 
5203 597
K_ THU·ªòC MINH 
5745 599
T√∫c T√¥ng Kh√¢m Ho√†ng ƒê·∫ø 
2425 697
Chi√™u T√¥ng Th·∫ßn Ho√†ng ƒê·∫ø 
13589 708
Cung Ho√†ng ƒê·∫ø 
11273 716
M·∫°c ƒêƒÉng Doanh 
1177 723
Trung T√¥ng Ho√†ng ƒê·∫ø 
456 743
Anh T√¥ng Tu·∫•n Ho√†ng ƒê·∫ø 
820 751
Th·∫ßn T√¥ng Uy√™n Ho√†ng ƒê·∫ø (Th∆∞·ª£ng) 
1175 817
Ch√¢n T√¥ng Thu·∫≠n Ho√†ng ƒê·∫ø 
1373 

In [752]:
z =''
for e in events:
    if len(e) < 50:
        print(e)
        z = e


[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  

[1a]  


In [753]:
events = [e for e in events if e !=z]
for e in events:
    if len(e) < 50:
        print(e)

In [754]:
len(events)

931

In [755]:
data = {
    "events": [event for event in events],
}

In [756]:
dfx = pd.DataFrame(data)

In [758]:
dfx.to_csv('history_events_process.csv',sep="\t", index=False)

In [759]:
df = pd.read_csv("history_events_process.csv", delimiter="\t")  # Ho·∫∑c sep="\t"
df

events
0    \n[1a] Tri·ªÅu Li·ªát ƒê·∫°i Phu, Qu·ªëc T·ª≠ Gi√°m T∆∞ Ngh...
1    K·ª∑ H·ªìng B√†ng Th·ªã \nKinh D∆∞∆°ng V∆∞∆°ng \n[1b] T√™n...
2    K·ª∑ H·ªìng B√†ng Th·ªã \nKinh D∆∞∆°ng V∆∞∆°ng \n[1b] T√™n...
3    H√πng V∆∞∆°ng \n[3a] Con L·∫°c Long Qu√¢n (kh√¥ng r√µ ...
4    K·ª∑ Nh√† Th·ª•c \nAn D∆∞∆°ng V∆∞∆°ng \nH·ªç Th·ª•c, t√™n h√∫...
..                                                 ...
926  Canh Th√¨n, [D∆∞∆°ng Ho√†] nƒÉm th·ª© 6 [1640], (Minh...
927  Nh√¢m Ng·ªç, [D∆∞∆°ng Ho√†] nƒÉm th·ª© 8 [1642], (Minh ...
928  Qu√Ω M√πi, [D∆∞∆°ng Ho√†] nƒÉm th·ª© 9 [1643], (t·ª´ th√°...
929  [·∫§t D·∫≠u, Th·ªãnh ƒê·ª©c nƒÉm th·ª© 3 [1655], (Minh L·ªãc...
930  B√≠nh Th√¢n, [Th·ªãnh ƒê·ª©c] nƒÉm th·ª© 4 [1656], (Minh...

[931 rows x 1 columns]

In [761]:
len(events)

931

In [803]:
def clean_text(text):
    lines = text.split('\n')
    content = lines[0]
    for line in lines[1:]:
        if content[-2:] =='. ':
            content =content + '\n' +line
        else: 
            content += line
    return content


z = clean_text(events[3])

H√πng V∆∞∆°ng 
g 
#####
H√πng V∆∞∆°ng [3a] Con L·∫°c Long Qu√¢n (kh√¥ng r√µ t√™n h√∫y)5, ƒë√≥ng ƒë√¥ ·ªü Phong Ch√¢u (nay l√† huy·ªán B·∫°ch H·∫°c)6. 
H√πng V∆∞∆°ng l√™n ng√¥i, ƒë·∫∑t qu·ªëc hi·ªáu l√† VƒÉn Lang (n∆∞·ªõc n√†y ƒë√¥ng gi√°p bi·ªÉn Nam H·∫£i, t√¢y ƒë·∫øn Ba 
a 
#####
H√πng V∆∞∆°ng [3a] Con L·∫°c Long Qu√¢n (kh√¥ng r√µ t√™n h√∫y)5, ƒë√≥ng ƒë√¥ ·ªü Phong Ch√¢u (nay l√† huy·ªán B·∫°ch H·∫°c)6. 
H√πng V∆∞∆°ng l√™n ng√¥i, ƒë·∫∑t qu·ªëc hi·ªáu l√† VƒÉn Lang (n∆∞·ªõc n√†y ƒë√¥ng gi√°p bi·ªÉn Nam H·∫£i, t√¢y ƒë·∫øn Ba Th·ª•c, b·∫Øc ƒë·∫øn h·ªì ƒê·ªông ƒê√¨nh, nam gi√°p n∆∞·ªõc H·ªì T√¥n, t·ª©c n∆∞·ªõc Chi√™m Th√†nh, nay l√† Qu·∫£ng Nam), chia 
a 
#####
H√πng V∆∞∆°ng [3a] Con L·∫°c Long Qu√¢n (kh√¥ng r√µ t√™n h√∫y)5, ƒë√≥ng ƒë√¥ ·ªü Phong Ch√¢u (nay l√† huy·ªán B·∫°ch H·∫°c)6. 
H√πng V∆∞∆°ng l√™n ng√¥i, ƒë·∫∑t qu·ªëc hi·ªáu l√† VƒÉn Lang (n∆∞·ªõc n√†y ƒë√¥ng gi√°p bi·ªÉn Nam H·∫£i, t√¢y ƒë·∫øn Ba Th·ª•c, b·∫Øc ƒë·∫øn h·ªì ƒê·ªông ƒê√¨nh, nam gi√°p n∆∞·ªõc H·ªì T√¥n, t·ª©c n∆∞·ªõc 

## 3. Text analysis

In [888]:
data_df = pd.read_csv('process2.csv', delimiter="\t")
data_df

events
0     \n[1a] Tri·ªÅu Li·ªát ƒê·∫°i Phu, Qu·ªëc T·ª≠ Gi√°m T∆∞ Ngh...
1     K·ª∑ H·ªìng B√†ng\n Th·ªã Kinh D∆∞∆°ng V∆∞∆°ng [1b] T√™n h...
2     L·∫°c Long Qu√¢n [2a]\n T√™n h√∫y l√† S√πng L√£m, con ...
3     S·ª≠ th·∫ßn Ng√¥ Sƒ© Li√™n n√≥i: Khi tr·ªùi ƒë·∫•t m·ªõi m·ªü m...
4     H√πng V∆∞∆°ng [3a] \nCon L·∫°c Long Qu√¢n (kh√¥ng r√µ ...
...                                                 ...
2209  M√πa xu√¢n, xu·ªëng chi·∫øu ƒëi kinh l∆∞·ª£c Thu·∫≠n Ho√°, ...
2210  M√πa ƒë√¥ng, th√°ng 10, ng√†y 13 l√† ng√†y Qu√Ω D·∫≠u, [...
2211  [·∫§t D·∫≠u, Th·ªãnh ƒê·ª©c nƒÉm th·ª© 3 [1655], (Minh L·ªãc...
2212  B√≠nh Th√¢n, [Th·ªãnh ƒê·ª©c] nƒÉm th·ª© 4 [1656], (Minh...
2213  M√πa xu√¢n, th√°ng gi√™ng, qu√¢n Thu·∫≠n H√≥a ƒë√°nh √∫p ...

[2214 rows x 1 columns]

In [870]:
sample = data_df['events'].values[:100]
event = data_df['events'].values

### 3.1 Ph√¢n t√≠ch t·ª´ kh√≥a

Ph√¢n t√≠ch t·ª´ kh√≥a, x√°c ƒë·ªãnh trong 1 s·ª± ki·ªán th∆∞·ªùng xu·∫•t hi·ªán c√°c t·ª´ kh√≥a n√†o, ƒë·ªÉ t·∫°o tags cho s·ª± ki·ªán

In [922]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Danh s√°ch t·ª´ d·ª´ng ti·∫øng Vi·ªát (c√≥ th·ªÉ m·ªü r·ªông)
stopwords_vn = [
    "l√†", "c·ªßa", "v√†", "v·ªõi", "ƒë∆∞·ª£c", "cho", "c√≥", "ƒë·∫øn", "t·ª´", "trong", "vi·ªác","b·ªçn",
     "nƒÉm", "th√°ng", "ng√†y", "vi·ªác", "ƒë√£", "b·ªã", "c≈©ng", "khi", "m√†", "th√¨","ƒë·ªÅu", "c√°c", "th·ª©", "li√™n", "ng√¥", "sƒ©",
    "h∆∞u", "m√πa", "bi·∫øt", "nh∆∞", "10", "11"
]

# D·ªØ li·ªáu vƒÉn b·∫£n
data = event

# Kh·ªüi t·∫°o TfidfVectorizer v·ªõi bigram v√† lo·∫°i b·ªè t·ª´ d·ª´ng
vectorizer = TfidfVectorizer(
    stop_words=stopwords_vn,  # Lo·∫°i b·ªè t·ª´ d·ª´ng
    ngram_range=(2, 4),       # Tr√≠ch xu·∫•t bigram (c·ª•m t·ª´ 2 t·ª´)
    max_features=1000           # L·∫•y 10 c·ª•m ph·ªï bi·∫øn nh·∫•t
)

X = vectorizer.fit_transform(data)

# L·∫•y danh s√°ch t·ª´ kh√≥a (bigrams) v√† tr·ªçng s·ªë TF-IDF
feature_names = vectorizer.get_feature_names_out()
frequencies = X.toarray().sum(axis=0)

# T·∫°o DataFrame ƒë·ªÉ hi·ªÉn th·ªã k·∫øt qu·∫£
df_freq = pd.DataFrame({'T·ª´ kh√≥a': feature_names, 'Tr·ªçng s·ªë TF-IDF': frequencies})
print(df_freq.sort_values(by='Tr·ªçng s·ªë TF-IDF', ascending=False)[:50])


           T·ª´ kh√≥a  Tr·ªçng s·ªë TF-IDF
882    xu·ªëng chi·∫øu        52.417602
561      qu·∫≠n c√¥ng        51.043781
879     xu√¢n gi√™ng        48.568760
702   th∆∞·ª£ng ho√†ng        42.805805
61     chi√™m th√†nh        41.361528
29         b·∫•y gi·ªù        40.276545
894       ƒëem qu√¢n        37.060444
454       nh√† minh        34.477430
903        ƒëi ƒë√°nh        34.280970
162     ho√†ng th√°i        30.951951
689        th√°i t·ª≠        30.849371
680       th√°i h·∫≠u        30.547939
168       ho√†ng ƒë·∫ø        30.141339
480      ni√™n hi·ªáu        29.973415
708       th·∫ßn n√≥i        28.655274
630        s·ª≠ th·∫ßn        28.650671
631    s·ª≠ th·∫ßn n√≥i        28.029889
459       nh√† t·ªëng        26.134155
476      nh·∫≠t th·ª±c        25.979482
442         ng·ª± s·ª≠        25.874579
321       l√™n ng√¥i        24.987718
844        vua sai        24.530679
782         tr·ªü v·ªÅ        24.236051
95         con g√°i        24.129198
703     th∆∞·ª£ng th∆∞ 

### 3.2 Ph√¢n t√≠ch ch·ªß ƒë·ªÅ

Ph√¢n t√≠ch ch·ªß ƒë·ªÅ, x√°c ƒë·ªãnh c√°c ch·ªß ƒë·ªÅ th∆∞·ªùng g·∫∑p, ƒë·ªÉ t·∫°o b·ªô ph√¢n lo·∫°i ch·ªß ƒë·ªÅ cho s·ª± ki·ªán

In [940]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

# Kh·ªüi t·∫°o LDA
lda = LatentDirichletAllocation(n_components=20)  # Gi·∫£ s·ª≠ c√≥ 3 ch·ªß ƒë·ªÅ
lda.fit(X)

# Hi·ªÉn th·ªã c√°c t·ª´ kh√≥a quan tr·ªçng cho m·ªói ch·ªß ƒë·ªÅ
for i, topic in enumerate(lda.components_):
    print(f"Ch·ªß ƒë·ªÅ {i}:")
    print([vectorizer.get_feature_names_out()[index] for index in topic.argsort()[-10:]])

Ch·ªß ƒë·ªÅ 0:
['quan qu√¢n', 'ƒëu·ªïi theo', 'r·∫•t nhi·ªÅu', 'an ph·ªß', 'ƒë√°nh gi·∫∑c', 'qu√¢n minh', 'qu√¢n gi·∫∑c', 'qu√¢n ƒë√°nh', 'c·ª≠a bi·ªÉn', 'ƒëem qu√¢n']
Ch·ªß ƒë·ªÅ 1:
['t√¢u l√™n', 'quan vi√™n', 'ph·ªß huy·ªán', 'ch·ªâ r·∫±ng', 'ƒë√¥ ƒë·ªëc', 'ra s·∫Øc ch·ªâ', 'ra s·∫Øc', 'ng∆∞·ªùi n√†o', 'nha m√¥n', 's·∫Øc ch·ªâ']
Ch·ªß ƒë·ªÅ 2:
['sai quan', 'minh vƒ©nh', 'an v∆∞∆°ng', 'b√¨nh an v∆∞∆°ng', 'b√¨nh an', 'thu·∫≠n thi√™n', 'kh√¥ng r√µ', 'r√µ t√™n', 'kh√¥ng r√µ t√™n', 'xu·ªëng chi·∫øu']
Ch·ªß ƒë·ªÅ 3:
['ch√≠nh s·ª±', 'an ph·ªß s·ª©', 'an ph·ªß', 'vua t·ªëng', 'chuy·ªÉn v·∫≠n', 'ng∆∞·ªùi h·ªç', 'n·ªôi m·∫≠t', 'm·∫≠t vi·ªán', 'th√°nh nh√¢n', 'nh·ªØ h√†i']
Ch·ªß ƒë·ªÅ 4:
['s·ª© sang nh√†', 'sang c·ªëng', 'sang nh√†', 'n∆∞·ªõc chi√™m', 'thu vua', 'sai s·ª© sang', 's·ª© sang', 'chi√™m th√†nh', 'nh√† t·ªëng', 'sai s·ª©']
Ch·ªß ƒë·ªÅ 5:
['xu·ªëng chi·∫øu', 'ban t∆∞·ªõc', 'v∆∞∆°ng h·∫ßu', 'm·ªôt l·∫ßn', 'l√†m s·ª≠', 'ƒë·ªãnh l·ªá', 'ƒë·∫°i phu', 'c√¥ng ch√∫a', 'vi·ªát v∆∞∆°ng', 'anh

### 3.3 Ph√¢n t√≠ch ƒë·ªô d√†i

Ph√¢n t√≠ch ƒë·ªô d√†i c·ªßa m·ªói s·ª± ki·ªán, ƒë·ªÉ x√°c ƒë·ªãnh ƒë∆∞·ª£c ƒë·ªô d√†i cho vi·ªác chunking

In [932]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# T√≠nh ƒë·ªô d√†i c·ªßa m·ªói ƒëo·∫°n text (s·ªë t·ª´)
lengths = [len(text.split()) for text in data]

# T√≠nh trung b√¨nh, min, max
average_length = np.mean(lengths)
min_length = np.min(lengths)
max_length = np.max(lengths)

# ‚úÖ V·∫Ω histogram ph√¢n b·ªë ƒë·ªô d√†i ƒëo·∫°n vƒÉn b·∫£n
plt.figure(figsize=(10, 5))
sns.histplot(lengths, bins=10, kde=True, color='skyblue')
plt.axvline(average_length, color='red', linestyle='dashed', label=f'Trung b√¨nh: {average_length:.2f}')
plt.axvline(min_length, color='green', linestyle='dashed', label=f'Th·∫•p nh·∫•t: {min_length}')
plt.axvline(max_length, color='purple', linestyle='dashed', label=f'Cao nh·∫•t: {max_length}')
plt.xlabel("S·ªë t·ª´ trong ƒëo·∫°n vƒÉn")
plt.ylabel("T·∫ßn su·∫•t")
plt.title("Ph√¢n b·ªë ƒë·ªô d√†i ƒëo·∫°n vƒÉn b·∫£n")
plt.legend()
plt.show()



In [939]:
np_len = np.array(lengths)
print((np_len < 200).sum())
print((np_len < 300).sum())
print((np_len < 400).sum())
print((np_len < 500).sum())


1614
1856
1949
2019


## 4. Create dataset and embeddings


### 4.1 X√°c ƒë·ªãnh th·ªùi gian s·ª± ki·ªán

In [891]:
import re

def extract_years(text):
    pattern = r'\[(-?\d{2,4})\s*TCN?\]|\[(-?\d{2,4})\]|/(-?\d{2,4})/'
    matches = re.findall(pattern, text)
    
    years = []
    for match in matches:
        if match[0]:  # NƒÉm c√≥ "TCN", l∆∞u v·ªõi d·∫•u √¢m
            years.append(f"-{match[0]}")
        elif match[1]:  # NƒÉm b√¨nh th∆∞·ªùng trong []
            years.append(match[1])
        elif match[2]:  # NƒÉm trong //
            years.append(match[2])
    if years: return years[0]
    else: return pd.NA

data_df['year'] = data_df['events'].apply(extract_years)

In [1072]:
extract_years('[2012 TCN]')

'-2012'

In [1170]:
data_df

events  year
0     \n[1a] Tri·ªÅu Li·ªát ƒê·∫°i Phu, Qu·ªëc T·ª≠ Gi√°m T∆∞ Ngh...   NaN
1     K·ª∑ H·ªìng B√†ng\n Th·ªã Kinh D∆∞∆°ng V∆∞∆°ng [1b] T√™n h...   NaN
2     L·∫°c Long Qu√¢n [2a]\n T√™n h√∫y l√† S√πng L√£m, con ...   NaN
3     S·ª≠ th·∫ßn Ng√¥ Sƒ© Li√™n n√≥i: Khi tr·ªùi ƒë·∫•t m·ªõi m·ªü m...   NaN
4     H√πng V∆∞∆°ng [3a] \nCon L·∫°c Long Qu√¢n (kh√¥ng r√µ ...   NaN
...                                                 ...   ...
2209  M√πa xu√¢n, xu·ªëng chi·∫øu ƒëi kinh l∆∞·ª£c Thu·∫≠n Ho√°, ...  <NA>
2210  M√πa ƒë√¥ng, th√°ng 10, ng√†y 13 l√† ng√†y Qu√Ω D·∫≠u, [...  <NA>
2211  [·∫§t D·∫≠u, Th·ªãnh ƒê·ª©c nƒÉm th·ª© 3 [1655], (Minh L·ªãc...  1655
2212  B√≠nh Th√¢n, [Th·ªãnh ƒê·ª©c] nƒÉm th·ª© 4 [1656], (Minh...  1656
2213  M√πa xu√¢n, th√°ng gi√™ng, qu√¢n Thu·∫≠n H√≥a ƒë√°nh √∫p ...  <NA>

[2214 rows x 2 columns]

In [1075]:
data_df['year'][:20]

0     <NA>
1     <NA>
2     <NA>
3     <NA>
4     <NA>
5     <NA>
6     <NA>
7     <NA>
8     <NA>
9     -258
10    <NA>
11    -257
12    <NA>
13    -255
14    <NA>
15    -249
16    -221
17    -214
18    -210
19    -208
Name: year, dtype: object

### 4.2 Ph√¢n lo·∫°i s·ª± ki·ªán (topic)

In [991]:
from collections import defaultdict

def classify_event(text):
    """
    Ph√¢n lo·∫°i s·ª± ki·ªán trong vƒÉn b·∫£n d·ª±a tr√™n c√°c t·ª´ kh√≥a.
    
    Args:
        text (str): VƒÉn b·∫£n ƒë·∫ßu v√†o.
    
    Returns:
        list: Danh s√°ch c√°c lo·∫°i s·ª± ki·ªán t√¨m th·∫•y trong vƒÉn b·∫£n.
    """
    categories = {
    "Ngo·∫°i giao": ["sai s·ª©", "sang c·ªëng", "ƒë√°p l·ªÖ", "sang phong", "sang ch·∫ßu", "bi·∫øu nh√†", "li√™n minh", "giao h·∫£o", "ngh·ªã h√≤a", "h√≤a ∆∞·ªõc", "tri·ªÅu c·ªëng", "giao th∆∞∆°ng", "ti·∫øp s·ª©", "h·ªôi ki·∫øn", "s·ª© th·∫ßn", "k·∫øt th√¥ng gia"],
    "Ch√≠nh tr·ªã": ["ƒë·ªïi ni√™n hi·ªáu", "h·∫° l·ªánh", "xu·ªëng chi·∫øu", "s·∫Øc ch·ªâ", "phong", "ni√™n hi·ªáu", "ra l·ªánh", "lu·∫≠t", "l√™n ng√¥i", "phong vua", "t·∫•u l√™n", "sai quan", "b√£i nhi·ªám", "c·∫£i c√°ch", "s·∫Øc l·ªánh","phong t∆∞·ªõc","th√°i t·ª≠", "c√¥ng ch√∫a","ho√†ng h·∫≠u", "th√°i h·∫≠u"],
    "VƒÉn h√≥a": ["thi h·ªôi", "ƒëua thuy·ªÅn", "ch√πa", "ƒë·∫°o", "ƒë·ªÅn", "mi·∫øu", "l·ªÖ", "h·ªçc", "tr·ª• tr√¨", "di·ªÖn", "m·ªü h·ªôi", "h√°t tu·ªìng", "m√∫a r·ªëi", "th∆∞ ph√°p", "vƒÉn bia"],
    "Gi√°o d·ª•c": [ "kinh s·ª≠","l·∫•y ƒë·ªó", "l·∫•y sinh", "h·ªçc sinh", "ng∆∞·ªùi ƒë·ªó", "tuy·ªÉn ch·ªçn nh√¢n t√†i", "khoa c·ª≠", "m·ªü tr∆∞·ªùng", "d·∫°y h·ªçc", "ti·∫øn sƒ©", "tr·∫°ng nguy√™n", "b·∫£ng v√†ng","s√°ch", "in s√°ch", "d·ªãch s√°ch"],
    "Qu√¢n s·ª±": ["ƒëi ƒë√°nh", "l√†m ph·∫£n", "l√†m lo·∫°n", "ƒë√°nh chi·∫øm", "t∆∞·ªõng qu√¢n", "ƒëem qu√¢n", "gi·∫∑c", "tr·∫≠n", "chi·∫øn th·∫Øng", "thua", "t·∫•n c√¥ng", "qu√¢n gi·∫∑c", "qu√¢n ƒë√°nh", "b·∫Øt s·ªëng", "chi·∫øm ƒë√≥ng", "v√¢y th√†nh", "ph·ª•c k√≠ch", "b·ªë tr·∫≠n", "ƒë·∫°o qu√¢n", "ƒë√≥ng gi·ªØ"],
    "Th·ªùi ti·∫øt": ["h·∫°n", "l·ª•t", "b√£o", "ƒë·ªông ƒë·∫•t", "sao ch·ªïi", "s·∫•m", "m∆∞a", "d·ªãch", "nh·∫≠t th·ª±c", "nguy·ªát th·ª±c", "n∆∞·ªõc to", "m·∫•t m√πa", "n·∫Øng g·∫Øt", "tuy·∫øt", "bƒÉng gi√°"],
    "ƒê·ªëi tho·∫°i": ["n√≥i", "r·∫±ng", "l√†m th∆°", "ngh·ªã lu·∫≠n", "tranh bi·ªán"],
    "Gi·ªõi thi·ªáu nh√¢n v·∫≠t": ["t√™n hu√Ω l√†", "·ªü ng√¥i", "cha l√†", "xu·∫•t th√¢n", "t√¥n x∆∞ng", "danh hi·ªáu"],
    "Kinh t·∫ø - T√†i ch√≠nh": ["b·∫°c", "v√†ng", "ti·ªÅn ƒë·ªìng", "ph√°t h√†nh", "thu·∫ø", "mi·ªÖn thu·∫ø", "ƒë·ªãnh gi√°", "gi√° c·∫£", "bu√¥n b√°n", "th∆∞∆°ng m·∫°i" ,],
    "X√£ h·ªôi - ƒê·ªùi s·ªëng": ["c·ª©u t·∫ø", "ph√°t ch·∫©n", "y t·∫ø", "b·ªánh d·ªãch", "thu·ªëc", "ch·ªØa b·ªánh", "c·ª©u ƒë√≥i", "tr·ª£ c·∫•p", "gi·∫£m t√¥", "gi·∫£m thu·∫ø","c√†y","ru·ªông ƒë·∫•t", "kho l√∫a"],
    "Ki·∫øn tr√∫c - X√¢y d·ª±ng": ["x√¢y th√†nh", "ƒë·∫Øp ƒë√™", "l√†m c·∫ßu", "s·ª≠a ch√πa", "x√¢y cung", "l√†m ƒë∆∞·ªùng", "d·ª±ng bia", "kh·∫Øc ch·ªØ","x√¢y ch√πa"],
    "Khoa h·ªçc - K·ªπ thu·∫≠t": ["sao tr·ªùi", "thi√™n vƒÉn", "t√≠nh l·ªãch", "ƒëo ƒë·∫°c", "y thu·∫≠t", "ph√°t minh"]
}
    
    event_types = []
    text = text.lower()  
    
    for category, keywords in categories.items():
        for keyword in keywords:
            if keyword in text:
                event_types.append(category)
                break  # Ch·ªâ c·∫ßn t√¨m th·∫•y 1 t·ª´ kh√≥a l√† ƒë·ªß ƒë·ªÉ ph√¢n lo·∫°i v√†o nh√≥m ƒë√≥

    return event_types

# üìå V√≠ d·ª• s·ª≠ d·ª•ng
text ='''
Gi√°p Ng·ªç, /·ª®ng Thi√™n/ nƒÉm th·ª© 1 /994/, (T·ªëng Thu·∫ßn Ho√° nƒÉm th·ª© 5). M√πa xu√¢n, th√°ng gi√™ng, ƒë·ªïi ni√™n hi·ªáu. 
Phong ho√†ng t·ª≠ th·ª´ m∆∞·ªùi l√† Mang l√†m Nam Qu·ªëc V∆∞∆°ng, ƒë√≥ng ·ªü ch√¢u V≈© Lung3. 
Sai nha hi·ªáu l√† Ph√≠ S√πng ƒê·ª©c sang nh√† T·ªëng [21a] sang thƒÉm ƒë√°p l·ªÖ. 
Ch√°u vua n∆∞·ªõc Chi√™m Th√†nh l√† Ch·∫ø Cai v√†o ch·∫ßu. Tr∆∞·ªõc ƒë√¢y, n∆∞·ªõc Chi√™m Th√†nh sai Ch·∫ø ƒê√¥ng d√¢ng s·∫£n v·∫≠t ƒë·ªãa ph∆∞∆°ng, vua tr√°ch l√† tr√°i l·ªÖ, kh√¥ng nh·∫≠n. Vua n∆∞·ªõc ƒë√≥ s·ª£, m·ªõi sai Ch·∫ø Cai v√†o ch·∫ßu.

'''
print(classify_event(text))  # Output: ['ch√≠nh tr·ªã', 'vƒÉn h√≥a']


['Ngo·∫°i giao', 'Ch√≠nh tr·ªã', 'VƒÉn h√≥a']


### 4.3 X√°c ƒë·ªãnh keywords (tags)

**X√°c ƒë·ªãnh t√™n ri√™ng (ƒê·ªãa danh, nh√¢n v·∫≠t):**

In [999]:
import re

def extract_name(text):   
    regex = r'[A-Z√Ä√Å·∫¢√É·∫†ƒÇ·∫Æ·∫∞·∫≤·∫¥·∫∂√Ç·∫§·∫¶·∫®·∫™·∫¨ƒê√à√â·∫∫·∫º·∫∏√ä·∫æ·ªÄ·ªÇ·ªÑ·ªÜ√å√ç·ªàƒ®·ªä√í√ì·ªé√ï·ªå√î·ªê·ªí·ªî·ªñ·ªò∆†·ªö·ªú·ªû·ª†·ª¢√ô√ö·ª¶≈®·ª§∆Ø·ª®·ª™·ª¨·ªÆ·ª∞·ª≤√ù·ª∂·ª∏·ª¥][a-z√†√°·∫£√£·∫°ƒÉ·∫Ø·∫±·∫≥·∫µ·∫∑√¢·∫•·∫ß·∫©·∫´·∫≠ƒë√®√©·∫ª·∫Ω·∫π√™·∫ø·ªÅ·ªÉ·ªÖ·ªá√¨√≠·ªâƒ©·ªã√≤√≥·ªè√µ·ªç√¥·ªë·ªì·ªï·ªó·ªô∆°·ªõ·ªù·ªü·ª°·ª£√π√∫·ªß≈©·ª•∆∞·ª©·ª´·ª≠·ªØ·ª±·ª≥√Ω·ª∑·ªπ·ªµ]+(?:\s+[A-Z√Ä√Å·∫¢√É·∫†ƒÇ·∫Æ·∫∞·∫≤·∫¥·∫∂√Ç·∫§·∫¶·∫®·∫™·∫¨ƒê√à√â·∫∫·∫º·∫∏√ä·∫æ·ªÄ·ªÇ·ªÑ·ªÜ√å√ç·ªàƒ®·ªä√í√ì·ªé√ï·ªå√î·ªê·ªí·ªî·ªñ·ªò∆†·ªö·ªú·ªû·ª†·ª¢√ô√ö·ª¶≈®·ª§∆Ø·ª®·ª™·ª¨·ªÆ·ª∞·ª≤√ù·ª∂·ª∏·ª¥][a-z√†√°·∫£√£·∫°ƒÉ·∫Ø·∫±·∫≥·∫µ·∫∑√¢·∫•·∫ß·∫©·∫´·∫≠ƒë√®√©·∫ª·∫Ω·∫π√™·∫ø·ªÅ·ªÉ·ªÖ·ªá√¨√≠·ªâƒ©·ªã√≤√≥·ªè√µ·ªç√¥·ªë·ªì·ªï·ªó·ªô∆°·ªõ·ªù·ªü·ª°·ª£√π√∫·ªß≈©·ª•∆∞·ª©·ª´·ª≠·ªØ·ª±·ª≥√Ω·ª∑·ªπ·ªµ]+)+'
    
    name_raw = re.findall(regex, text)
    
    name_result = []
    for name in name_raw:
        # Lo·∫°i b·ªè d·∫•u c√¢u ·ªü cu·ªëi n·∫øu c√≥
        name_final = re.sub(r'[\.,!?;:]$', '', name)
        name_result.append(name_final)
    
    return set(name_result)

In [1000]:
# V√≠ d·ª•
text = "M√πa xu√¢n, th√°ng gi√™ng, vua Nh√† L√Ω sai s·ª© sang phong. Th√°ng hai, ƒë√°nh gi·∫∑c ·ªü √Åi Ch√¢u, chi·∫øn th·∫Øng vang d·ªôi."
result = extract_name(text)
print(result)

{'√Åi Ch√¢u', 'Nh√† L√Ω'}


**Tri·ªÅu ƒë·∫°i:**

In [ ]:
trieudai_vn = [
    "Nh√† ƒêinh", "nh√† ƒêinh", "Vua ƒêinh", "vua ƒêinh", 
    "Nh√† Ti·ªÅn L√™", "nh√† Ti·ªÅn L√™", "Vua L√™", "vua L√™",  
    "Nh√† L√™", "nh√† L√™", 
    "Nh√† L√Ω", "nh√† L√Ω", "Vua L√Ω", "vua L√Ω",  
    "Nh√† Tr·∫ßn", "nh√† Tr·∫ßn", "Vua Tr·∫ßn", "vua Tr·∫ßn",  
    "Nh√† H·ªì", "nh√† H·ªì", "Vua H·ªì", "vua H·ªì",  
    "Nh√† H·∫≠u L√™", "nh√† H·∫≠u L√™",
    "Nh√† Nguy·ªÖn", "nh√† Nguy·ªÖn", "Vua Nguy·ªÖn", "vua Nguy·ªÖn"
]

# Tri·ªÅu ƒë·∫°i Trung Qu·ªëc (c√≥ th·ªÉ g·ªçi b·∫±ng "Nh√†", "qu√¢n", "gi·∫∑c")
trieudai_tq = [
    "Nh√† T·∫ßn", "nh√† T·∫ßn", "qu√¢n T·∫ßn", "gi·∫∑c T·∫ßn",  
    "Nh√† H√°n", "nh√† H√°n", "qu√¢n H√°n", "gi·∫∑c H√°n",  
    "Nh√† ƒê∆∞·ªùng", "nh√† ƒê∆∞·ªùng", "qu√¢n ƒê∆∞·ªùng", "gi·∫∑c ƒê∆∞·ªùng", 'vua ƒê∆∞·ªùng',  
    "Nh√† T·ªëng", "nh√† T·ªëng", "qu√¢n T·ªëng", "gi·∫∑c T·ªëng", 'vua T·ªëng',
    "Nh√† Nguy√™n", "nh√† Nguy√™n", "qu√¢n Nguy√™n", "gi·∫∑c Nguy√™n",  'vua Nguy√™n', 
    "Nh√† Minh", "nh√† Minh", "qu√¢n Minh", "gi·∫∑c Minh", 'vua Minh',
    "Nh√† Thanh", "nh√† Thanh", "qu√¢n Thanh", "gi·∫∑c Thanh"
]

trieudai = set(trieudai_vn + trieudai_tq)

In [986]:
text = "M√πa xu√¢n, th√°ng gi√™ng, vua Nh√† L√Ω sai s·ª© sang phong. Th√°ng hai, ƒë√°nh gi·∫∑c qu√¢n Nguy√™n ·ªü √Åi Ch√¢u, chi·∫øn th·∫Øng vang d·ªôi."

found_dynasties = {dyn for dyn in trieudai if dyn in text}
found_dynasties

{'Nh√† L√Ω', 'qu√¢n Nguy√™n'}

**S·ª± ki·ªán:**

In [987]:
keywords = set([
    "sai s·ª©", "sang c·ªëng", "ƒë√°p l·ªÖ", "sang phong", "sang ch·∫ßu", "bi·∫øu nh√†", "li√™n minh", "giao h·∫£o", "ngh·ªã h√≤a", "h√≤a ∆∞·ªõc", "tri·ªÅu c·ªëng", "giao th∆∞∆°ng", "ti·∫øp s·ª©", "h·ªôi ki·∫øn", "s·ª© th·∫ßn", "k·∫øt th√¥ng gia",  # Ngo·∫°i giao
    "ƒë·ªïi ni√™n hi·ªáu", "h·∫° l·ªánh", "xu·ªëng chi·∫øu", "s·∫Øc ch·ªâ", "phong", "ni√™n hi·ªáu", "ra l·ªánh", "lu·∫≠t", "l√™n ng√¥i", "phong vua", "t·∫•u l√™n", "sai quan", "b√£i nhi·ªám", "c·∫£i c√°ch", "s·∫Øc l·ªánh","phong t∆∞·ªõc","th√°i t·ª≠", "c√¥ng ch√∫a","ho√†ng h·∫≠u", "th√°i h·∫≠u",  # Ch√≠nh tr·ªã
    "thi h·ªôi", "ƒëua thuy·ªÅn", "ch√πa", "ƒë·∫°o", "ƒë·ªÅn", "mi·∫øu", "l·ªÖ", "h·ªçc", "tr·ª• tr√¨", "di·ªÖn", "m·ªü h·ªôi", "h√°t tu·ªìng", "m√∫a r·ªëi", "th∆∞ ph√°p", "vƒÉn bia",  # VƒÉn h√≥a
    "kinh s·ª≠","l·∫•y ƒë·ªó", "l·∫•y sinh", "h·ªçc sinh", "ng∆∞·ªùi ƒë·ªó", "tuy·ªÉn ch·ªçn nh√¢n t√†i", "khoa c·ª≠", "m·ªü tr∆∞·ªùng", "d·∫°y h·ªçc", "ti·∫øn sƒ©", "tr·∫°ng nguy√™n", "b·∫£ng v√†ng","s√°ch", "in s√°ch", "d·ªãch s√°ch",  # Gi√°o d·ª•c
    "ƒëi ƒë√°nh", "l√†m ph·∫£n", "l√†m lo·∫°n", "ƒë√°nh chi·∫øm", "t∆∞·ªõng qu√¢n", "ƒëem qu√¢n", "gi·∫∑c", "tr·∫≠n", "chi·∫øn th·∫Øng", "thua", "t·∫•n c√¥ng", "qu√¢n gi·∫∑c", "qu√¢n ƒë√°nh", "b·∫Øt s·ªëng", "chi·∫øm ƒë√≥ng", "v√¢y th√†nh", "ph·ª•c k√≠ch", "b·ªë tr·∫≠n", "ƒë·∫°o qu√¢n", "ƒë√≥ng gi·ªØ",  # Chi·∫øn tranh
    "h·∫°n", "l·ª•t", "b√£o", "ƒë·ªông ƒë·∫•t", "sao ch·ªïi", "s·∫•m", "m∆∞a", "d·ªãch", "nh·∫≠t th·ª±c", "nguy·ªát th·ª±c", "n∆∞·ªõc to", "m·∫•t m√πa", "n·∫Øng g·∫Øt", "tuy·∫øt", "bƒÉng gi√°",  # Th·ªùi ti·∫øt
    "n√≥i", "r·∫±ng", "l√†m th∆°", "ngh·ªã lu·∫≠n", "tranh bi·ªán",  # ƒê·ªëi tho·∫°i
    "t√™n hu√Ω l√†", "·ªü ng√¥i", "cha l√†", "xu·∫•t th√¢n", "t√¥n x∆∞ng", "danh hi·ªáu",
    "b·∫°c", "v√†ng", "ti·ªÅn ƒë·ªìng", "ph√°t h√†nh", "thu·∫ø", "mi·ªÖn thu·∫ø", "ƒë·ªãnh gi√°", "gi√° c·∫£", "bu√¥n b√°n", "th∆∞∆°ng m·∫°i" ,
    "c·ª©u t·∫ø", "ph√°t ch·∫©n", "y t·∫ø", "b·ªánh d·ªãch", "thu·ªëc", "ch·ªØa b·ªánh", "c·ª©u ƒë√≥i", "tr·ª£ c·∫•p", "gi·∫£m t√¥", "gi·∫£m thu·∫ø","c√†y","ru·ªông ƒë·∫•t", "kho l√∫a",
    "x√¢y th√†nh", "ƒë·∫Øp ƒë√™", "l√†m c·∫ßu", "s·ª≠a ch√πa", "x√¢y cung", "l√†m ƒë∆∞·ªùng", "d·ª±ng bia", "kh·∫Øc ch·ªØ","x√¢y ch√πa",
    "sao tr·ªùi", "thi√™n vƒÉn", "t√≠nh l·ªãch", "ƒëo ƒë·∫°c", "y thu·∫≠t", "ph√°t minh"
])

In [990]:
text = "M√πa xu√¢n, th√°ng gi√™ng, vua Nh√† L√Ω sai s·ª© sang phong. Th√°ng hai, ƒë√°nh gi·∫∑c qu√¢n Nguy√™n ·ªü √Åi Ch√¢u, chi·∫øn th·∫Øng vang d·ªôi."
text_lower = text.lower()
found_keywords = {kw for kw in keywords if kw in text_lower}
found_keywords

{'chi·∫øn th·∫Øng', 'gi·∫∑c', 'phong', 'sai s·ª©', 'sang phong'}

**T·∫°o tags:**

In [1001]:
def extract_tags(text):
    text_lower = text.lower()
    named_entities = extract_name(text)
    found_dynasties = {dyn for dyn in trieudai if dyn in text}
    event_keywords = {kw for kw in keywords if kw in text_lower}
    
    return event_keywords | named_entities | found_dynasties
    

In [1100]:
text = "M√πa xu√¢n, th√°ng gi√™ng, vua Nh√† L√Ω sai s·ª© sang phong. Th√°ng hai, ƒë√°nh gi·∫∑c qu√¢n Nguy√™n ·ªü √Åi Ch√¢u, chi·∫øn th·∫Øng vang d·ªôi."
print(classify_event(text))
extract_tags(text)

['Ngo·∫°i giao', 'Ch√≠nh tr·ªã', 'Qu√¢n s·ª±']


{'Nh√† L√Ω',
 'chi·∫øn th·∫Øng',
 'gi·∫∑c',
 'phong',
 'qu√¢n Nguy√™n',
 'sai s·ª©',
 'sang phong',
 '√Åi Ch√¢u'}

### 4.4 Chunk

In [1057]:
import re
import nltk
from nltk.tokenize import sent_tokenize

# T·∫£i resource c·∫ßn thi·∫øt cho NLTK
def setup_nltk():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

# H√†m t·∫°o chunk th√¥ng minh d·ª±a tr√™n c√°c c√¢u ho√†n ch·ªânh
def create_chunks(text, max_chunk_size=200, min_chunk_size=50):
    """
    T·∫°o c√°c chunk th√¥ng minh d·ª±a tr√™n ranh gi·ªõi c√¢u v√† ƒëo·∫°n.
    
    Args:
        text (str): VƒÉn b·∫£n ƒë·∫ßu v√†o.
        max_chunk_size (int): S·ªë t·ª´ t·ªëi ƒëa trong m·ªôt chunk.
        min_chunk_size (int): S·ªë t·ª´ t·ªëi thi·ªÉu ƒë·ªÉ t·∫°o th√†nh m·ªôt chunk ƒë·ªôc l·∫≠p.
    
    Returns:
        list: Danh s√°ch c√°c chunk c√≥ n·ªôi dung ng·ªØ nghƒ©a h·ª£p l√Ω.
    """
    setup_nltk()
    
    # T√°ch ƒëo·∫°n vƒÉn (paragraph)
    paragraphs = re.split(r'\n\s*\n', text)
    chunks = []
    current_chunk = []
    current_chunk_size = 0
    
    for paragraph in paragraphs:
        # B·ªè qua ƒëo·∫°n tr·ªëng
        if not paragraph.strip():
            continue
            
        # T√°ch c√¢u trong ƒëo·∫°n
        sentences = sent_tokenize(paragraph)
        
        paragraph_added = False
        
        for sentence in sentences:
            sentence_words = sentence.split()
            sentence_length = len(sentence_words)
            
            # Tr∆∞·ªùng h·ª£p 1: C√¢u qu√° d√†i, bu·ªôc ph·∫£i t√°ch ra th√†nh chunk ri√™ng
            if sentence_length > max_chunk_size:
                # X·ª≠ l√Ω chunk hi·ªán t·∫°i n·∫øu c√≥
                if current_chunk:
                    chunks.append(' '.join(current_chunk))
                    current_chunk = []
                    current_chunk_size = 0
                
                # T√°ch c√¢u d√†i th√†nh nhi·ªÅu chunk
                for i in range(0, sentence_length, max_chunk_size):
                    if i + max_chunk_size >= sentence_length:
                        # Ph·∫ßn cu·ªëi c·ªßa c√¢u d√†i
                        if current_chunk_size == 0:
                            # Th√™m tr·ª±c ti·∫øp v√†o chunks n·∫øu ch∆∞a c√≥ g√¨ trong current_chunk
                            chunks.append(' '.join(sentence_words[i:]))
                        else:
                            # Th√™m v√†o chunk hi·ªán t·∫°i
                            current_chunk.extend(sentence_words[i:])
                            current_chunk_size += len(sentence_words[i:])
                    else:
                        # Ph·∫ßn gi·ªØa c·ªßa c√¢u d√†i
                        chunks.append(' '.join(sentence_words[i:i+max_chunk_size]))
                        
            # Tr∆∞·ªùng h·ª£p 2: C√¢u v·ª´a + chunk hi·ªán t·∫°i s·∫Ω v∆∞·ª£t qu√° max_chunk_size
            elif current_chunk_size + sentence_length > max_chunk_size:
                # Ho√†n th√†nh chunk hi·ªán t·∫°i
                chunks.append(' '.join(current_chunk))
                
                # B·∫Øt ƒë·∫ßu chunk m·ªõi v·ªõi c√¢u hi·ªán t·∫°i
                current_chunk = sentence_words
                current_chunk_size = sentence_length
                paragraph_added = True
                
            # Tr∆∞·ªùng h·ª£p 3: Th√™m c√¢u v√†o chunk hi·ªán t·∫°i
            else:
                current_chunk.extend(sentence_words)
                current_chunk_size += sentence_length
                paragraph_added = True
        
        # K·∫øt th√∫c ƒëo·∫°n, th√™m d·∫•u xu·ªëng d√≤ng n·∫øu kh√¥ng ph·∫£i l√† ƒëo·∫°n cu·ªëi
        if paragraph_added and current_chunk:
            current_chunk[-1] = current_chunk[-1] + '.'
            
    # X·ª≠ l√Ω chunk cu·ªëi c√πng n·∫øu c√≥
    if current_chunk:
        # N·∫øu chunk cu·ªëi qu√° nh·ªè v√† c√≥ chunk tr∆∞·ªõc ƒë√≥, g·ªôp v·ªõi chunk cu·ªëi
        if current_chunk_size < min_chunk_size and len(chunks) > 0:
            last_chunk = chunks[-1].split()
            combined_size = len(last_chunk) + current_chunk_size
            
            if combined_size <= max_chunk_size * 1.2:  # Cho ph√©p v∆∞·ª£t m·ªôt ch√∫t
                chunks[-1] = ' '.join(last_chunk + current_chunk)
            else:
                chunks.append(' '.join(current_chunk))
        else:
            chunks.append(' '.join(current_chunk))
    
    return chunks


In [1060]:
chunk = create_chunks('''T√¢n S·ª≠u, [H·ªìng ƒê·ª©c] nƒÉm th·ª© 2 [1481], (Minh Th√†nh H√≥a nƒÉm th·ª© 17). M√πa xu√¢n, th√°ng gi√™ng, ng√†y 19, ƒê√¥ c·∫•p s·ª± trung V≈© M·ªông Khang d√¢ng th∆∞ n√≥i c√°c vi·ªác, ƒë·∫°i kh√°i c√≥ 4 ƒëi·ªÅu: 1- K·∫ª n√†o x√¢m chi·∫øm ru·ªông ƒë·∫•t c·ªßa ng∆∞·ªùi kh√°c v√† ch·∫∑t n√™u, nh·ªï m·ªëc, t·ª± ti·ªán l·∫≠p gi·ªõi h·∫°n th√¨ ph·∫£i ph·∫°t tr∆∞·ª£ng v√† bi·∫øm tru·∫•t: n·∫øu ph√° b·ªè b·ªù ru·ªông th√¨ ch·ªâ ph·∫°t ti·ªÅn th√¥i. 
2- K·∫ª n√†o ch·∫∑t ph√° c√¢y c·ªëi, tre pheo ·ªü v∆∞·ªùn m·ªô ng∆∞·ªùi kh√°c th√¨ ph·∫£i ph·∫°t tr∆∞·ª£ng v√† bi·∫øm tru·∫•t, n·∫øu c√†y ph√° m·ªô c·ªßa ng∆∞·ªùi x∆∞a th√¨ ƒë·ªÅu x·ª≠ t·ªôi l∆∞u.  3- L·∫•y v·ª£ ph·∫£i l·∫•y ng∆∞·ªùi ngo√†i, kh√¥ng ƒë∆∞·ª£c l·∫•y [30a] k·∫ª gi√†u sang, th√¢n th√≠ch phi lo·∫°i, k·∫ª n√†o vi ph·∫°m, th√¨ x·ª≠ t·ªôi ƒë·ªì. 
4- C√°c quan ty s·ªü t·∫°i khi kh√°m x√©t √°n ki·ªán trong ngo√†i v√† lo·∫°i t·∫°p ph·∫°m, ph·∫£i theo ƒë√∫ng h·∫°n ƒë·ªãnh trong lu·∫≠t m√† x·ª≠ ƒëo√°n ph·∫£i tr√°i, kh√¥ng ƒë∆∞·ª£c ƒë√πn ƒë·∫©y cho nhau ƒë·ªÉ k√©o d√†i; k·∫ª n√†o l√†m tr√°i th√¨ ph·∫£i ph·∫°t ti·ªÅn nh∆∞ l·ªánh. 
M√πa h·∫°, th√°ng 4, thi h·ªôi cho c√°c c·ª≠ nh√¢n trong n∆∞·ªõc, l·∫•y ƒë·ªó b·ªçn Ph·∫°m ƒê√¥n L·ªÖ 40 ng∆∞·ªùi. 
Ng√†y 27, vua ng·ª± ƒëi·ªán K√≠nh Thi√™n, th√¢n h√†nh ra ƒë·∫ßu b√†i vƒÉn s√°ch h·ªèi v·ªÅ l√Ω s·ªë. 
Cho b·ªçn Ph·∫°m ƒê√¥n L·ªÖ, L∆∞u H∆∞ng Hi·∫øu, Nguy·ªÖn Do√£n ƒê·ªãnh ba ng∆∞·ªùi ƒë·ªï ti·∫øn sƒ© c·∫≠p ƒë·ªá; b·ªçn Ng√¥ VƒÉn C·∫£nh 8 ng∆∞·ªùi ƒë·ªó ti·∫øn sƒ© xu·∫•t th√¢n; b·ªçn Nguy·ªÖn Minh ƒê·∫°o 29 ng∆∞·ªùi ƒë·ªó ƒë·ªìng ti·∫øn sƒ© xu·∫•t th√¢n. 
(Ph·∫°m ƒê√¥n L·ªÖ, t√™n t·ª± l√† L∆∞ Khanh, ng∆∞·ªùi huy·ªán Ng·ª± Thi√™n, ph·ªß T√¢n H∆∞ng ng·ª• ·ªü l√†ng Thanh Nh√†n, huy·ªán Kim Hoa2, khi ƒë·ªó 27 tu·ªïi, thi h∆∞∆°ng, h·ªôi, ƒë√¨nh ƒë·ªÅu ƒë·ªó ƒë·∫ßu, ng∆∞·ªùi ƒë·ªùi g·ªçi l√† Tam nguy√™n ƒê√¥n L·ªÖ). 
Th√°ng 5, ng√†y 21, tri·ªáu b·ªçn ti·∫øn sƒ© Ph·∫°m ƒê√¥n L·ªÖ v√†o trong ƒêan Tr√¨. Vua [30b] ng·ª± ƒëi·ªán K√≠nh Thi√™n. C√°c quan H·ªìng l√¥ truy·ªÅn l·ªánh g·ªçi t√™n. L·∫°i b·ªô ban ·∫•n m·ªánh. L·ªÖ b·ªô b∆∞ng b·∫£ng v√†ng, n·ªïi tr·ªëng nh·∫°c, r∆∞·ªõc ra ngo√†i c·ª≠a ƒê√¥ng Hoa treo l√™n. Xong r·ªìi ty M√£ c·ª©u ƒëem ng·ª±a t·ªët ƒë∆∞a tr·∫°ng nguy√™n v·ªÅ nh√†. 
Xu·ªëng l·ªánh r·∫±ng: Quan c√°c v·ªá, ty, s·ªü Tr√°ng sƒ©, Th·∫ßn v≈©, Hi·ªáu l·ª±c, ƒêi·ªán ti·ªÅn, Ng≈© ph·ªß, M√£ nh√†n, Thu·∫ßn t∆∞·ª£ng trong kinh, c√°c quan v·ªá s·ªü, ph·ªß, ch√¢u, huy·ªán b√™n ngo√†i, n·∫øu c√≥ ng∆∞·ªùi ƒë·∫Ωo kho√©t qu√¢n l√≠nh, m·ªçt h·∫°i nh√¢n d√¢n, ch·ªâ chƒÉm l√†m l·ª£i cho m√¨nh, kh√¥ng nghƒ© ƒë·∫øn ph√©p n∆∞·ªõc, ·ªü trong th√¨ ƒë∆∞·ªùng quan l√† c√°c Ch·ªâ huy, ƒë·ªÅ ƒë·ªëc Ki·ªÉm ƒëi·ªÉm, ƒê√¥ ƒë·ªëc  v√† c√°c quan ƒêo√°n s·ª±, Kinh l·ªãch; b√™n ngo√†i th√¨ hay ty Th·ª´a, Hi·∫øn, ƒë·ªÅu ph·∫£i c√¥ng b·∫±ng x√©t x·ª≠, tham kh·∫£o d∆∞ lu·∫≠n c·ªßa m·ªçi ng∆∞·ªùi, ng∆∞·ªùi n√†o tr∆∞·ªõc kia c√≥ nh≈©ng lo·∫°n, tuy kh√¥ng c√≥ ch·ª©ng th·ª±c, nh∆∞ng m·ªçi ng∆∞·ªùi ƒë·ªÅu bi·∫øt; c√πng l√† ng∆∞·ªùi n√†o li√™m khi·∫øt, kh√¥ng m·∫Øc [31a] th√≥i tham √¥, k√™ ra t·ª´ng lo·∫°i t√¢u l√™n c·∫£, trao cho Gi√°m s√°t ng·ª± s·ª≠ ·ªü ƒë·∫°o ƒë√≥ th·∫©m tra l·∫°i, l√†m b·∫£n t√¢u l√™n, s·∫Ω khu x·ª≠ ƒë·ªÉ t·ªè r√µ khuy·∫øn kh√≠ch v√† tr·ª´ng ph·∫°t. 
L·∫≠p s·ªü ƒê·ªìn ƒëi·ªÅn. Xu·ªëng chi·∫øu r·∫±ng: M·ªü ƒë·ªìn ƒëi·ªÅn l√† ƒë·ªÉ d√πng h·∫øt ti·ªÅm l·ª±c c·ªßa ngh·ªÅ n√¥ng, m·ªü r·ªông ngu·ªìn tr√≠ch tr·ªØ cho nh√† n∆∞·ªõc. Nay l·ªánh cho c√°c x·ª© ƒë·ªãnh ƒë·ªìn ƒëi·ªÅn th√†nh ba b·∫≠c th∆∞·ª£ng, trung, h·∫°. 
Th√°ng 6, ng√†y 20, ch·ªçn th·∫£i b·ªõt quan vi√™n v√† b·ªïng l·ªôc. 
Quan vi√™n qu√° nhi·ªÅu, ti√™u ph√≠ l·ªôc kho, l·ª•c khoa ph·∫£i tra x√©t xem t·ª´ nƒÉm Quang Thu·∫≠n th·ª© 2 ƒë·∫øn nay, quan vi√™n n√†o ƒë√£ t·ª´ng ph·∫°m c√°c t·ªôi h·ªëi l·ªô, ƒë√£ b·ªã x·ª≠ c√°c t·ªôi bi·∫øm, gi√°ng, c√πng c√°c t∆∞·ªõng hi·ªáu, qu·∫£n √°p, ƒë·ªÉ thi·∫øu v√°n thuy·ªÅn, g·ªó l·∫°t, c·ªßi, g·∫°ch... v·ª•ng tr·ªôm b·∫Øt qu√¢n n·ªôp ti·ªÅn, ƒë·∫øn n·ªói thi·∫øu nhi·ªÅu th·ª© ph·∫£i n·ªôp, tr·ªã gi√° t·ª´ 10 quan tr·ªü l√™n m√† ng∆∞·ªùi ·∫•y h√£y c√≤n t·∫°i ch·ª©c th√¨ b·∫Øt ph·∫£i th√¥i vi·ªác, ƒë·ªÉ tri·ªát nh·ªØng quan tham nh≈©ng [31b] cho b·ªõt l·ªôc. 
Ng√†y 21, xu·ªëng chi·∫øu r·∫±ng: C√°c quan trong ngo√†i ƒëua nhau ki·∫øm l·ª£i l√† b·ªüi quan c√≥ tr√°ch nhi·ªám ƒÉn n√≥i ch∆∞a ƒë∆∞·ª£c ng∆∞·ªùi gi·ªèi. 
K·∫ª th√¨ nhu nh∆° kh√¥ng c√≥ t√†i c√°n g√¨, ng∆∞·ªùi th√¨ ƒë·∫£ k√≠ch qu√° ƒë·ªÉ r∆∞·ªõc gi√®m pha. Quan l·∫°i tham nh≈©ng, d√¢n ch√∫ng o√°n th√°n, ƒëem kh√≠ d·ªØ l√†m tr√°i kh√≠ h√≤a, m·ªëi t·ªá n√†y ph·∫£i n√™n tr·ª´ b·ªè. T·ª´ nay tr·ªü ƒëi, c√°c ch·ª©c khoa ƒë√†i, hi·∫øn s√°t, tri·ªÅu th·∫ßn ph·∫£i c√¥ng b·∫±ng m√† b√†n ch·ªçn, ai n√™n th·∫£ ra, ai n√™n b·ªï thay, l√†m b·∫£n t√¢u l√™n thi h√†nh, ƒë·ªÉ tr·ª´ b·ªè t·ªá c≈©. 
M√πa thu, th√°ng 7, ng√†y m·ªìng 4, Ph√≥ ƒë√¥ ng·ª± s·ª≠ ki√™m T·∫£ xu√¢n ph∆∞·ªùng t√° trung do√£n Qu√°ch ƒê√¨nh B·∫£o t√¢u v·ªÅ vi·ªác t·ª± ti·ªán ƒëu·ªïi d√¢n tr√∫ ng·ª• nh∆∞ sau: "Th·∫ßn tr·ªôm nghƒ© r·∫±ng: Kinh s∆∞ l√† g·ªëc c·ªßa b·ªën ph∆∞∆°ng; ti·ªÅn c·ªßa trao ƒë·ªïi mua b√°n t·∫•t ph·∫£i cho l∆∞u th√¥ng ƒë·ªß d√πng, kh√¥ng n√™n ƒë·ªÉ thi·∫øu th·ªën. Tr∆∞·ªõc ƒë√¢y, d√¢n c∆∞ ph·ªß Ph·ª•ng Thi√™n, tr·ª´ nh·ªØng ng∆∞·ªùi qu√™ qu√°n ·ªü ph·ªß ƒë√≥, gi√°n ho·∫∑c c√≥ ng∆∞·ªùi tuy kh√¥ng ph·∫£i qu√™ ·ªü ƒë√≥, nh∆∞ng c√≥ [32a] c·ª≠a hi·ªáu, thu·∫ø ng·∫°ch v√† n·ªôp thu·∫ø, ch·ªãu sai d·ªãch v·ªõi b·∫£n ph∆∞·ªùng. Nay quan ph·ªß Ph·ª•ng Thi√™n l·∫°i kh√¥ng h·ªèi xem d√¢n t·∫°p c∆∞ ·ªü ƒë√≥ c√≥ c·ª≠a hi·ªáu, thu·∫ø ng·∫°ch hay kh√¥ng, ƒëu·ªïi h·∫øt c·∫£ v·ªÅ nguy√™n qu√°n, e r·∫±ng nh∆∞ th·∫ø th√¨ n∆°i kinh s∆∞ s·∫Ω bu√¥n b√°n th∆∞a th·ªõt, kh√¥ng c√≤n s·∫ßm u·∫•t ph·ªìn th·ªãnh n·ªØa, kh√¥ng nh·ªØng ng∆∞·ªùi l√†m ngh·ªÅ bu√¥n b√°n s·∫Ω th·∫•t nghi·ªáp nhi·ªÅu, m√† ch·ª£ b√∫a e s·∫Ω tr·ªëng r·ªóng, ng·∫°ch thu·∫ø s·∫Ω c√≥ th·ªÉ thi·∫øu h·ª•t, c√≥ ph·∫ßn kh√¥ng ti·ªán. V√¨ th·∫ø t√¢u xin: Ngo√†i nh·ªØng k·∫ª v√¥ lo√†i t·∫°p c∆∞ th√¨ n√™n ƒëu·ªïi ƒëi, c√≤n nh·ªØng ng∆∞·ªùi chuy√™ c√≥ h√†ng ch·ª£, c·ª≠a hi·ªáu, tr∆∞·ªõc ƒë√¢y ƒë√£ bi√™n v√†o thu·∫ø ng·∫°ch th√¨ h√£y cho ƒë∆∞·ª£c c∆∞ tr√∫ ƒë·ªÉ bu√¥n b√°n sinh nhai, cho v√†o b·∫£n ph∆∞·ªùng n·ªôp thu·∫ø theo l·ªá c≈©". 
Th√°ng 9, ng√†y 27, ra s·∫Øc ch·ªâ r·∫±ng: T·ª´ nay, c√°c phi t·∫ßn v√† quan vi√™n th√¨ l√†m s·ªï ri√™ng. 
M√πa ƒë√¥ng, th√°ng 10, ra s·∫Øc ch·ªâ r·∫±ng: Quan c√°c v·ªá, s·ªü, ng∆∞·ªùi n√†o d√°m h·∫°ch [32b] s√°ch ti·ªÅn c·ªßa nh∆∞ tr∆∞·ªõc, t√≠nh t·ª´ 5 ti·ªÅn tr·ªü l√™n th√¨ ph√°p ty s·∫Ω tr·ªã t·ªôi theo lu·∫≠t ph√°p, ngo√†i ra, t·ª´ 4 ti·ªÅn tr·ªü xu·ªëng th√¨ nh·∫•t lu·∫≠t b√£i ch·ª©c sung qu√¢n, nh∆∞ l·ªá quan vi√™n tham nh≈©ng. 
Xu·ªëng ch·ªâ d·ª• g·ªìm 56 ƒëi·ªÅu: Tuy·ªÉn ng∆∞·ªùi b·ªï sung qu√¢n ng≈©. 
Kh·∫£o thi con ch√°u c√°c quan vi√™n, h·ªèi v·ªÅ l√†m vƒÉn, vi·∫øt ch·ªØ, l√†m to√°n. 
ƒê√†o h·ªì H·∫£i Tr√¨. H·ªì n√†y quanh co ƒë·∫øn 100 d·∫∑m. Gi·ªØa h·ªì c√≥ ƒëi·ªán Th√∫y Ng·ªçc, b√™n h·ªì x√¢y ƒëi·ªán Gi·∫£ng V√µ ƒë·ªÉ t·∫≠p luy·ªán binh t∆∞·ª£ng. 
L·∫•y Nguy·ªÖn B√° K√Ω l√†m L·∫°i b·ªô th∆∞·ª£ng th∆∞. (Nguy·ªÖn B√° K√Ω ng∆∞·ªùi Ch√≠ Linh).
''')

In [1061]:
for c in chunk:
    print(c)
    print('########')

T√¢n S·ª≠u, [H·ªìng ƒê·ª©c] nƒÉm th·ª© 2 [1481], (Minh Th√†nh H√≥a nƒÉm th·ª© 17). M√πa xu√¢n, th√°ng gi√™ng, ng√†y 19, ƒê√¥ c·∫•p s·ª± trung V≈© M·ªông Khang d√¢ng th∆∞ n√≥i c√°c vi·ªác, ƒë·∫°i kh√°i c√≥ 4 ƒëi·ªÅu: 1- K·∫ª n√†o x√¢m chi·∫øm ru·ªông ƒë·∫•t c·ªßa ng∆∞·ªùi kh√°c v√† ch·∫∑t n√™u, nh·ªï m·ªëc, t·ª± ti·ªán l·∫≠p gi·ªõi h·∫°n th√¨ ph·∫£i ph·∫°t tr∆∞·ª£ng v√† bi·∫øm tru·∫•t: n·∫øu ph√° b·ªè b·ªù ru·ªông th√¨ ch·ªâ ph·∫°t ti·ªÅn th√¥i. 2- K·∫ª n√†o ch·∫∑t ph√° c√¢y c·ªëi, tre pheo ·ªü v∆∞·ªùn m·ªô ng∆∞·ªùi kh√°c th√¨ ph·∫£i ph·∫°t tr∆∞·ª£ng v√† bi·∫øm tru·∫•t, n·∫øu c√†y ph√° m·ªô c·ªßa ng∆∞·ªùi x∆∞a th√¨ ƒë·ªÅu x·ª≠ t·ªôi l∆∞u. 3- L·∫•y v·ª£ ph·∫£i l·∫•y ng∆∞·ªùi ngo√†i, kh√¥ng ƒë∆∞·ª£c l·∫•y [30a] k·∫ª gi√†u sang, th√¢n th√≠ch phi lo·∫°i, k·∫ª n√†o vi ph·∫°m, th√¨ x·ª≠ t·ªôi ƒë·ªì. 4- C√°c quan ty s·ªü t·∫°i khi kh√°m x√©t √°n ki·ªán trong ngo√†i v√† lo·∫°i t·∫°p ph·∫°m, ph·∫£i theo ƒë√∫ng h·∫°n ƒë·ªãnh trong lu·∫≠t m√† x·ª≠ ƒëo√°n ph·∫£i tr√°i, kh√¥ng ƒë∆∞·ª£c ƒë√πn ƒë·∫©y 

### 4.5 T·∫°o embeddings:

In [1081]:
from sentence_transformers import SentenceTransformer, models
import numpy as np

def create_embeddings(chunks, model_name="vinai/phobert-base-v2"):
    """
    T·∫°o embeddings cho c√°c ƒëo·∫°n vƒÉn b·∫£n s·ª≠ d·ª•ng PhoBERT v·ªõi mean pooling.
    
    Args:
        chunks (list): Danh s√°ch c√°c ƒëo·∫°n vƒÉn b·∫£n.
        model_name (str): T√™n c·ªßa m√¥ h√¨nh PhoBERT.
    
    Returns:
        numpy.ndarray: Ma tr·∫≠n embeddings.
    """
    # Load PhoBERT v·ªõi transformer model
    word_embedding_model = models.Transformer(model_name, max_seq_length=256)

    # Th√™m l·ªõp mean pooling
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode_mean_tokens=True)

    # K·∫øt h·ª£p th√†nh m·ªôt SentenceTransformer model
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    # T·∫°o embeddings
    embeddings = model.encode(chunks, show_progress_bar=True)

    return np.array(embeddings)  # Chuy·ªÉn v·ªÅ numpy array

In [1083]:
create_embeddings('''
M√πa h·∫°, th√°ng 4, thi h·ªôi cho c√°c c·ª≠ nh√¢n trong n∆∞·ªõc, l·∫•y ƒë·ªó b·ªçn Ph·∫°m ƒê√¥n L·ªÖ 40 ng∆∞·ªùi. Ng√†y 27, vua ng·ª± ƒëi·ªán K√≠nh Thi√™n, th√¢n h√†nh ra ƒë·∫ßu b√†i vƒÉn s√°ch h·ªèi v·ªÅ l√Ω s·ªë. Cho b·ªçn Ph·∫°m ƒê√¥n L·ªÖ, L∆∞u H∆∞ng Hi·∫øu, Nguy·ªÖn Do√£n ƒê·ªãnh ba ng∆∞·ªùi ƒë·ªï ti·∫øn sƒ© c·∫≠p ƒë·ªá; b·ªçn Ng√¥ VƒÉn C·∫£nh 8 ng∆∞·ªùi ƒë·ªó ti·∫øn sƒ© xu·∫•t th√¢n; b·ªçn Nguy·ªÖn Minh ƒê·∫°o 29 ng∆∞·ªùi ƒë·ªó ƒë·ªìng ti·∫øn sƒ© xu·∫•t th√¢n. (Ph·∫°m ƒê√¥n L·ªÖ, t√™n t·ª± l√† L∆∞ Khanh, ng∆∞·ªùi huy·ªán Ng·ª± Thi√™n, ph·ªß T√¢n H∆∞ng ng·ª• ·ªü l√†ng Thanh Nh√†n, huy·ªán Kim Hoa2, khi ƒë·ªó 27 tu·ªïi, thi h∆∞∆°ng, h·ªôi, ƒë√¨nh ƒë·ªÅu ƒë·ªó ƒë·∫ßu, ng∆∞·ªùi ƒë·ªùi g·ªçi l√† Tam nguy√™n ƒê√¥n L·ªÖ). Th√°ng 5, ng√†y 21, tri·ªáu b·ªçn ti·∫øn sƒ© Ph·∫°m ƒê√¥n L·ªÖ v√†o trong ƒêan Tr√¨. Vua [30b] ng·ª± ƒëi·ªán K√≠nh Thi√™n. C√°c quan H·ªìng l√¥ truy·ªÅn l·ªánh g·ªçi t√™n. L·∫°i b·ªô ban ·∫•n m·ªánh. L·ªÖ b·ªô b∆∞ng b·∫£ng v√†ng, n·ªïi tr·ªëng nh·∫°c, r∆∞·ªõc ra ngo√†i c·ª≠a ƒê√¥ng Hoa treo l√™n. Xong r·ªìi ty M√£ c·ª©u ƒëem ng·ª±a t·ªët ƒë∆∞a tr·∫°ng nguy√™n v·ªÅ nh√†.
''').shape

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(768,)

### 4.6 Process dataset:

In [1163]:
from tqdm import tqdm
from datetime import datetime


def process_dataset(data, chunk_size=200, model_name="vinai/phobert-base-v2"):
    """
    X·ª≠ l√Ω to√†n b·ªô dataset l·ªãch s·ª≠ ƒë·ªÉ chu·∫©n b·ªã cho RAG.
    
    Args:
        data (list): Dataset ch·ª©a c√°c record l·ªãch s·ª≠.
        chunk_size (int): K√≠ch th∆∞·ªõc t·ªëi ƒëa c·ªßa m·ªói ƒëo·∫°n.
        model_name (str): T√™n c·ªßa m√¥ h√¨nh sentence transformer.
    
    Returns:
        dict: D·ªØ li·ªáu ƒë√£ ƒë∆∞·ª£c x·ª≠ l√Ω v√† t·∫°o embeddings.
    """
    # Kh·ªüi t·∫°o c√°c list ƒë·ªÉ l∆∞u tr·ªØ k·∫øt qu·∫£
    all_chunks = []
    all_metadata = []
    original_indices = []
        
    records = data
    
    # X·ª≠ l√Ω t·ª´ng record
    current_year = '-258'
    for idx, record in tqdm(enumerate(records), total=len(records), desc="Processing records"):
        text = record
        if not text:
            continue
        
        # Tr√≠ch xu·∫•t metadata
        event_types = classify_event(text)
        tags = extract_tags(text)
        years = extract_year(text)
        if not years:
            years = current_year
        else:
            current_year = years
            
        if not tags:
            tags = ['l·ªãch s·ª≠']
        if not event_types:
            event_types = ['l·ªãch s·ª≠']
        
        # T·∫°o chunk t·ª´ vƒÉn b·∫£n
        chunks = create_chunks(text, chunk_size)
        
        # Th√™m th√¥ng tin v√†o danh s√°ch
        for chunk in chunks:
            all_chunks.append(chunk)
            metadata = {
                'original_index': idx,
                'event_types': event_types,
                'tags': list(tags),  # Chuy·ªÉn set th√†nh list ƒë·ªÉ d·ªÖ serialize
                'years': years,
                'chunk_text': chunk
            }
            
            all_metadata.append(metadata)
            original_indices.append(idx)
    
    # T·∫°o embeddings
    print("ƒêang t·∫°o embeddings...")
    embeddings = create_embeddings(all_chunks, model_name)
    
    # ƒê√≥ng g√≥i k·∫øt qu·∫£
    result = {
        'chunks': all_chunks,
        'metadata': all_metadata,
        #'embeddings': embeddings,
        'original_indices': original_indices,
        #'embedding_model': model_name,
        #'created_at': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'chunk_size': chunk_size,
        #'overlap': overlap
    }
    
    return result

In [1164]:
data

['M√πa xu√¢n, th√°ng gi√™ng, vua Nh√† L√Ω sai s·ª© sang phong. Th√°ng hai, ƒë√°nh gi·∫∑c ·ªü √Åi Ch√¢u, chi·∫øn th·∫Øng vang d·ªôi.',
 'ƒê·∫°i x√° thi√™n h·∫°, phong t∆∞·ªõc cho c√°c c√¥ng th·∫ßn. Nh√† ƒêinh l·∫≠p ni√™n hi·ªáu m·ªõi, ra l·ªánh m·ªü h·ªôi ƒëua thuy·ªÅn.',
 'NƒÉm ·∫•y, Nh·∫≠t th·ª±c x·∫£y ra. Nh√† T·ªëng sai s·ª© sang tri·ªÅu Nh√† H·∫≠u L√™, ƒë√°p l·ªÖ h·∫≠u hƒ©nh.']

In [1165]:
data_df['events'][100:105]

100    Tr·ªü l√™n l√† th·ªùi thu·ªôc H√°n, t·ª´ nƒÉm Qu√Ω M√£o ƒë·∫øn ...
101    K·ª∑ Sƒ© V∆∞∆°ng Sƒ© V∆∞∆°ng1 ·ªû ng√¥i 40 nƒÉm, th·ªç 90 tu...
102    ƒêinh M√£o, nƒÉm th·ª© 1 [187], (H√°n Trung B√¨nh nƒÉm...
103    Canh Th√¨n, nƒÉm th·ª© 14 [200], (H√°n Hi·∫øn ƒê·∫ø Hi·ªáp...
104    S·ª≠ th·∫ßn Ng√¥ Sƒ© Li√™n n√≥i:  Ng∆∞·ªùi qu√¢n t·ª≠ ƒë·ªëi v·ªõ...
Name: events, dtype: object

In [1166]:
process_dataset = process_dataset(data = data_df['events'][1:2].values, chunk_size = 300)


Processing records: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 1/1 [00:00<00:00, 998.64it/s]


ƒêang t·∫°o embeddings...


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [1159]:
d = 0
for item in process_dataset['metadata']:
    #print('type: ',item['event_types'])
    if not item['tags']:
        print(item)
        d+=1
    if not item['event_types']:
        print(item) 
    if not item['years']:
        print(item)
    if not item['chunk_text']:
        print(item)


In [1169]:
process_dataset

{'chunks': ['K·ª∑ H·ªìng B√†ng Th·ªã Kinh D∆∞∆°ng V∆∞∆°ng [1b] T√™n h√∫y l√† L·ªôc T·ª•c, con ch√°u h·ªç Th·∫ßn N√¥ng6. Nh√¢m Tu·∫•t, nƒÉm th·ª© 17. X∆∞a ch√°u ba ƒë·ªùi c·ªßa Vi√™m ƒê·∫ø h·ªç Th·∫ßn N√¥ng l√† ƒê·∫ø Minh sinh ra ƒê·∫ø Nghi, sau ƒê·∫ø Minh nh√¢n ƒëi tu·∫ßn ph∆∞∆°ng Nam, ƒë·∫øn Ng≈© Lƒ©nh8 l·∫•y con g√°i V·ª• Ti√™n, sinh ra vua [Kinh D∆∞∆°ng V∆∞∆°ng]. Vua l√† b·∫≠c th√°nh tr√≠ th√¥ng minh, ƒê·∫ø Minh r·∫•t y√™u qu√Ω, mu·ªën cho n·ªëi ng√¥i. Vua c·ªë nh∆∞·ªùng cho anh, kh√¥ng d√°m v√¢ng m·ªánh. ƒê·∫ø Minh m·ªõi l·∫≠p ƒê·∫ø Nghi l√† con n·ªëi ng√¥i, cai qu·∫£n ph∆∞∆°ng B·∫Øc, phong cho vua l√†m Kinh D∆∞∆°ng V∆∞∆°ng, cai qu·∫£n ph∆∞∆°ng Nam, g·ªçi l√† n∆∞·ªõc X√≠ch Qu·ª∑. Vua l·∫•y con g√°i ƒê·ªông ƒê√¨nh Qu√¢n t√™n l√† Th·∫ßn Long9 sinh ra L·∫°c Long Qu√¢n (X√©t: ƒê∆∞·ªùng k·ª∑ ch√©p: th·ªùi Kinh D∆∞∆°ng c√≥ ng∆∞·ªùi ƒë√†n b√† chƒÉn d√™, t·ª± x∆∞ng l√† con g√°i √∫t c·ªßa ƒê·ªông ƒê√¨nh Qu√¢n, l·∫•y con th·ª© c·ªßa Kinh Xuy√™n, b·ªã b·ªè, vi·∫øt th∆∞ nh·ªù Li·ªÖu Ngh·ªã t√¢

In [1167]:
import json

# Ghi v√†o file JSON
with open("data.json", "w", encoding="utf-8") as file:
    json.dump(process_dataset, file, indent=4, ensure_ascii=False)

print("ƒê√£ l∆∞u file JSON th√†nh c√¥ng!")


ƒê√£ l∆∞u file JSON th√†nh c√¥ng!


### 4.7 Test:

In [1114]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from rank_bm25 import BM25Okapi
import numpy as np

def preprocess_text(text):
    """Ti·ªÅn x·ª≠ l√Ω vƒÉn b·∫£n ƒë·ªÉ ph√π h·ª£p v·ªõi BM25"""
    return text.lower().split()

def search(query, processed_data, top_k=5, model_name=None, alpha=0.6):
    """
    T√¨m ki·∫øm c√°c ƒëo·∫°n vƒÉn b·∫£n li√™n quan b·∫±ng c√°ch k·∫øt h·ª£p BM25 v√† Cosine Similarity.

    Args:
        query (str): C√¢u truy v·∫•n.
        processed_data (dict): D·ªØ li·ªáu ƒë√£ ƒë∆∞·ª£c x·ª≠ l√Ω (ch·ª©a chunks, embeddings, metadata).
        top_k (int): S·ªë l∆∞·ª£ng k·∫øt qu·∫£ tr·∫£ v·ªÅ.
        model_name (str): T√™n m√¥ h√¨nh embedding. N·∫øu None, s·∫Ω d√πng m√¥ h√¨nh trong processed_data.
        alpha (float): Tr·ªçng s·ªë ƒë·ªÉ k·∫øt h·ª£p BM25 v√† Cosine Similarity (0.0: ch·ªâ BM25, 1.0: ch·ªâ Cosine).

    Returns:
        list: Danh s√°ch c√°c ƒëo·∫°n vƒÉn b·∫£n li√™n quan nh·∫•t k√®m theo metadata.
    """
    # X√°c ƒë·ªãnh m√¥ h√¨nh embedding
    if model_name is None:
        model_name = processed_data.get('embedding_model', "vinai/phobert-base-v2")

    # T·∫°o embedding cho c√¢u truy v·∫•n
    model = SentenceTransformer(model_name)
    query_embedding = model.encode([query])[0]

    # T√≠nh cosine similarity
    embeddings = processed_data['embeddings']
    cosine_scores = cosine_similarity([query_embedding], embeddings)[0]

    # T√≠nh ƒëi·ªÉm BM25
    tokenized_chunks = [preprocess_text(chunk) for chunk in processed_data['chunks']]
    bm25 = BM25Okapi(tokenized_chunks)
    bm25_scores = bm25.get_scores(preprocess_text(query))

    # Chu·∫©n h√≥a ƒëi·ªÉm v·ªÅ kho·∫£ng [0, 1]
    bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + 1e-9)
    cosine_scores = (cosine_scores - np.min(cosine_scores)) / (np.max(cosine_scores) - np.min(cosine_scores) + 1e-9)

    # K·∫øt h·ª£p ƒëi·ªÉm BM25 v√† Cosine Similarity
    combined_scores = alpha * cosine_scores + (1 - alpha) * bm25_scores

    # L·∫•y top_k k·∫øt qu·∫£
    top_indices = combined_scores.argsort()[-top_k:][::-1]

    results = []
    for idx in top_indices:
        result = {
            'chunk': processed_data['chunks'][idx],
            'metadata': processed_data['metadata'][idx],
            'similarity_score': combined_scores[idx],
            'cosine_score': cosine_scores[idx],
            'bm25_score': bm25_scores[idx],
        }
        results.append(result)

    return results


In [1116]:
query ='H√πng V∆∞∆°ng'

search(query,process_dataset)

No sentence-transformers model found with name vinai/phobert-base-v2. Creating a new one with mean pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'chunk': '[6a] Gi√°p Th√¨n, nƒÉm th·ª© 1 [257 TCN], (Chu No√£n V∆∞∆°ng nƒÉm th·ª© 58). Vua ƒë√£ th√¥n t√≠nh ƒë∆∞·ª£c n∆∞·ªõc VƒÉn Lang, ƒë·ªïi qu·ªëc hi·ªáu l√† √Çu L·∫°c. Tr∆∞·ªõc kia vua nhi·ªÅu l·∫ßn ƒëem qu√¢n ƒë√°nh H√πng V∆∞∆°ng, nh∆∞ng H√πng V∆∞∆°ng binh h√πng t∆∞·ªõng m·∫°nh, vua b·ªã thua m√£i. H√πng V∆∞∆°ng b·∫£o vua r·∫±ng: "Ta c√≥ s·ª©c th·∫ßn, n∆∞·ªõc Th·ª•c kh√¥ng s·ª£ ∆∞ ?" R·ªìi H√πng V∆∞∆°ng b·ªè kh√¥ng s·ª≠a sang v√µ b·ªã, ch·ªâ ham ƒÉn u·ªëng vui ch∆°i. Qu√¢n Th·ª•c k√©o s√°t ƒë·∫øn n∆°i, h√£y c√≤n say m·ªÅm ch∆∞a t·ªânh, r·ªìi th·ªï huy·∫øt nh·∫£y xu·ªëng gi·∫øng ch·∫øt, qu√¢n l√≠nh quay gi√°o ƒë·∫ßu h√†ng Th·ª•c V∆∞∆°ng..',
  'metadata': {'original_index': 10,
   'event_types': ['Qu√¢n s·ª±', 'ƒê·ªëi tho·∫°i'],
   'tags': ['VƒÉn Lang',
    'thua',
    '√Çu L·∫°c',
    'Chu No√£n V∆∞∆°ng',
    'Th·ª•c V∆∞∆°ng',
    'Qu√¢n Th·ª•c',
    'H√πng V∆∞∆°ng',
    'ƒëem qu√¢n',
    'R·ªìi H√πng V∆∞∆°ng',
    'r·∫±ng',
    'Gi√°p Th√¨n',
    'qu√¢n ƒë√°nh'],
   'years': '25